In [1]:
import pandas as pd
import numpy as np
import os
import sys
import dcanal
import static_frame
import statsmodels.api as sm
import statsmodels.duration.survfunc as sf
import statsmodels.duration.hazard_regression as coxph
import lifelines
from lifelines.datasets import load_rossi

In [3]:
#### Example
#### load example data from 
# rossi = load_rossi()
# cph = lifelines.CoxPHFitter()
# cph.fit(rossi, 'week', 'arrest')
# cph.predict_survival_function(rossi)

In [4]:
binary_abs_path = '/Users/ShaunPorwal/Documents/GitHub/dcanal/data/df_binary.csv'
surv_abs_path = '/Users/ShaunPorwal/Documents/GitHub/dcanal/data/df_surv.csv'
case_control_abs_path = '/Users/ShaunPorwal/Documents/GitHub/dcanal/data/df_case_control.csv'

df_binary = pd.read_csv(binary_abs_path)
df_surv = pd.read_csv(surv_abs_path)
df_case_control = pd.read_csv(case_control_abs_path)

In [5]:
cph = lifelines.CoxPHFitter()
cph_df = df_surv[['ttcancer','cancer','cancerpredmarker']]
cph.fit(cph_df, 'ttcancer', 'cancer')
# cph.print_summary()
new_cph_df = cph_df
new_cph_df['ttcancer'] = [1 for i in range(0,len(cph_df))]
cph_pred_vals = cph.predict_survival_function(new_cph_df, times = 1).values[0] #### all values in list of single list, so just dig em out with [0]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
# pd.Series(cph_pred_vals).describe()

In [7]:
# data = df_binary
# outcome = 'cancer'
# predictors = ['cancerpredmarker', 'marker']
# thresh_lo = 0.01
# thresh_hi = 0.99
# thresh_step = 0.01
# probabilities = [False, False]
# time = None
# prevalence = None
# pick_bin_surv = 'binary'

In [8]:

def dca(data,
       outcome,
       predictors,
       thresh_lo,
       thresh_hi,
       thresh_step,
       probabilities,
       time,
       prevalence,
       pick_bin_surv):

    
    
    
    '''
    Sequence of events
    1. convert to risk (convert to probabilities)
    2. calculate net benefit
        a. calculate_test_consequences for each predictor
        b. merge all dfs (one for each predictor)
        c. calculate net benefit based on other columns 
            i. nb = tpr - thresh / (1 - thresh) * fpr - harm
    '''
    
    model_frame = data[np.append(outcome, predictors)]

    for i in probabilities:
        continue


In [9]:
# binary inputs

binary_inputs = {
    
    'data': df_binary,
    'outcome': 'cancer',
    'predictors': ['cancerpredmarker', 'marker'],
    'thresh_lo': 0.01,
    'thresh_hi': 0.99,
    'thresh_step': 0.01,
    'probabilities': [False, True],
    'time': None,
    'prevalence': None,
    'pick_bin_surv': 'binary'
    
}



In [10]:
survival_inputs = {
    
    'data': df_binary,
    'outcome': 'cancer',
    'predictors': ['cancerpredmarker', 'marker'],
    'thresh_lo': 0.01,
    'thresh_hi': 0.99,
    'thresh_step': 0.01,
    'probabilities': [False, True],
    'time': None,
    'prevalence': None,
    'pick_bin_surv': 'binary'
    
}

In [11]:
# model_frame = data[np.append(outcome, predictors)] # get only cols from data that are in formula
# col_to_prob = model_frame.columns[1:][probabilities]
    

In [12]:




def convert_to_risk(model_frame, outcome, predictor, prevalence, time, pick_bin_surv):
    
    """Converts indicated predictor columns in dataframe into probabilities from 0 to 1
    
    Parameters
    ----------
    model_frame : dataframe
    outcome : string
    predictor : string
    prevalence : float
    time : float
    pick_bin_surv : string
    Returns 
    -------
    dataframe
    """
    
    if pick_bin_surv == 'binary':
        predicted_vals = sm.formula.glm(outcome + '~' + predictor, family=sm.families.Binomial(), 
                                            data=model_frame).fit().predict()
        model_frame[predictor] = predicted_vals
        return model_frame
    elif pick_bin_surv == 'survival':
        #### From lifelines dataframe
        cph = lifelines.CoxPHFitter()
        cph_df = model_frame[['ttcancer','cancer','cancerpredmarker']]
        cph.fit(cph_df, 'ttcancer', 'cancer')
        # cph.print_summary()
        new_cph_df = cph_df
        new_cph_df['ttcancer'] = [time for i in range(0,len(cph_df))]
        predicted_vals = cph.predict_survival_function(new_cph_df, times = time).values[0] #### all values in list of single list, so just dig em out with [0]
        new_model_frame = model_frame
        new_model_frame[predictor] = predicted_vals
        return new_model_frame


In [13]:
prob_converted_df = convert_to_risk(model_frame = df_surv, outcome = 'cancer', predictor = 'cancerpredmarker', prevalence = None, time = 1, pick_bin_surv = 'survival')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
prob_converted_df['cancerpredmarker'].describe()

count    7.500000e+02
mean     8.631214e-01
std      2.046330e-01
min      3.136229e-07
25%      8.934684e-01
50%      9.469032e-01
75%      9.573278e-01
max      9.621212e-01
Name: cancerpredmarker, dtype: float64

In [15]:
prob_model_frame = convert_to_risk(model_frame, col_to_prob, outcome, pick_bin_surv)

NameError: name 'model_frame' is not defined

In [16]:
binary_inputs['outcome']

'cancer'

In [17]:
model_frame = binary_inputs['data'][np.append(binary_inputs['outcome'], binary_inputs['predictors'])]

In [18]:
model_frame

,cancer,cancerpredmarker,marker
0,False,0.037201,0.776309
1,False,0.578907,0.267086
2,False,0.021551,0.169621
3,False,0.003910,0.023996
4,False,0.018790,0.070910
...,...,...,...
745,False,0.057813,0.654782
746,True,0.160424,1.030259
747,False,0.108838,0.151616
748,False,0.015285,0.624602


In [20]:
outcome_values

NameError: name 'outcome_values' is not defined

In [57]:
outcome_values = [val[0] for val in model_frame[[binary_inputs['outcome']]].values]
#risk_values = [val[0] for val in model_frame[[predictors[0]]].values] # get values of first predictor, list comprehension to unnest list of lists
predictor = 'cancerpredmarker'
thresholds = np.arange(binary_inputs['thresh_lo'], binary_inputs['thresh_hi'] + binary_inputs['thresh_step'], binary_inputs['thresh_step']) # array of values
# pick_bin_surv: variable that tells us the current setup is for binary or survival
prevalence = None
time = None

In [58]:
thresholds = np.insert(thresholds, 0, 0.1 ** 9)

In [18]:
df_binary

,patientid,cancer,dead,risk_group,age,famhistory,marker,cancerpredmarker
0,1,False,0,low,64.03445,0,0.776309,0.037201
1,2,False,0,high,78.46741,0,0.267086,0.578907
2,3,False,0,low,64.14617,0,0.169621,0.021551
3,4,False,0,low,58.53482,0,0.023996,0.003910
4,5,False,0,low,63.99250,0,0.070910,0.018790
...,...,...,...,...,...,...,...,...
745,746,False,0,intermediate,66.09628,0,0.654782,0.057813
746,747,True,0,intermediate,65.75370,1,1.030259,0.160424
747,748,False,0,intermediate,70.28287,0,0.151616,0.108838
748,749,False,1,low,61.32673,0,0.624602,0.015285


In [59]:
thresholds

array([1.0e-09, 1.0e-02, 2.0e-02, 3.0e-02, 4.0e-02, 5.0e-02, 6.0e-02,
       7.0e-02, 8.0e-02, 9.0e-02, 1.0e-01, 1.1e-01, 1.2e-01, 1.3e-01,
       1.4e-01, 1.5e-01, 1.6e-01, 1.7e-01, 1.8e-01, 1.9e-01, 2.0e-01,
       2.1e-01, 2.2e-01, 2.3e-01, 2.4e-01, 2.5e-01, 2.6e-01, 2.7e-01,
       2.8e-01, 2.9e-01, 3.0e-01, 3.1e-01, 3.2e-01, 3.3e-01, 3.4e-01,
       3.5e-01, 3.6e-01, 3.7e-01, 3.8e-01, 3.9e-01, 4.0e-01, 4.1e-01,
       4.2e-01, 4.3e-01, 4.4e-01, 4.5e-01, 4.6e-01, 4.7e-01, 4.8e-01,
       4.9e-01, 5.0e-01, 5.1e-01, 5.2e-01, 5.3e-01, 5.4e-01, 5.5e-01,
       5.6e-01, 5.7e-01, 5.8e-01, 5.9e-01, 6.0e-01, 6.1e-01, 6.2e-01,
       6.3e-01, 6.4e-01, 6.5e-01, 6.6e-01, 6.7e-01, 6.8e-01, 6.9e-01,
       7.0e-01, 7.1e-01, 7.2e-01, 7.3e-01, 7.4e-01, 7.5e-01, 7.6e-01,
       7.7e-01, 7.8e-01, 7.9e-01, 8.0e-01, 8.1e-01, 8.2e-01, 8.3e-01,
       8.4e-01, 8.5e-01, 8.6e-01, 8.7e-01, 8.8e-01, 8.9e-01, 9.0e-01,
       9.1e-01, 9.2e-01, 9.3e-01, 9.4e-01, 9.5e-01, 9.6e-01, 9.7e-01,
       9.8e-01, 9.9e

In [46]:
thresholds

array([1.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99])

In [ ]:
model_frame[[binary_inputs['outcome']]].values.flatten().tolist()


In [34]:
model_frame[[binary_inputs['outcome']]]

,cancer
0,False
1,False
2,False
3,False
4,False
...,...
745,False
746,True
747,False
748,False


In [558]:
outcome_values = [val[0] for val in model_frame[[binary_inputs['outcome']]].values]
#risk_values = [val[0] for val in model_frame[[predictors[0]]].values] # get values of first predictor, list comprehension to unnest list of lists
predictor = 'marker'
thresholds = np.arange(binary_inputs['thresh_lo'], binary_inputs['thresh_hi'] + binary_inputs['thresh_step'], binary_inputs['thresh_step']) # array of values
# pick_bin_surv: variable that tells us the current setup is for binary or survival
prevalence = None
time = None

In [559]:
df_binary.keys

<bound method NDFrame.keys of      patientid  cancer  dead    risk_group       age  famhistory    marker  \
0            1   False     0           low  64.03445           0  0.776309   
1            2   False     0          high  78.46741           0  0.267086   
2            3   False     0           low  64.14617           0  0.169621   
3            4   False     0           low  58.53482           0  0.023996   
4            5   False     0           low  63.99250           0  0.070910   
..         ...     ...   ...           ...       ...         ...       ...   
745        746   False     0  intermediate  66.09628           0  0.654782   
746        747    True     0  intermediate  65.75370           1  1.030259   
747        748   False     0  intermediate  70.28287           0  0.151616   
748        749   False     1           low  61.32673           0  0.624602   
749        750   False     0           low  61.66628           0  0.270679   

     cancerpredmarker  
0        

In [525]:
df_binary

,patientid,cancer,dead,risk_group,age,famhistory,marker,cancerpredmarker
0,1,False,0,low,64.03445,0,0.776309,0.037201
1,2,False,0,high,78.46741,0,0.267086,0.578907
2,3,False,0,low,64.14617,0,0.169621,0.021551
3,4,False,0,low,58.53482,0,0.023996,0.003910
4,5,False,0,low,63.99250,0,0.070910,0.018790
...,...,...,...,...,...,...,...,...
745,746,False,0,intermediate,66.09628,0,0.654782,0.057813
746,747,True,0,intermediate,65.75370,1,1.030259,0.160424
747,748,False,0,intermediate,70.28287,0,0.151616,0.108838
748,749,False,1,low,61.32673,0,0.624602,0.015285


In [550]:
n = len(outcome_values)
#df = pd.DataFrame({'threshold' : thresholds, 'n' : [n] * len(thresholds)})

In [1]:
prevalence_values = [pd.Series(outcome_values).value_counts()[1] / len(outcome_values)] * len(thresholds)


NameError: name 'pd' is not defined

In [552]:
#### Typical binary prevalence
#df.append({'prevalence' : prevalence_values})
df = pd.DataFrame({'threshold' : thresholds, 'n' : [n] * len(thresholds), 'prevalence' : prevalence_values})


In [560]:
#### If outcome is binary,

# Get test_pos_rate
count = 0
true_outcome = df_binary[df_binary['cancer'] == True][[predictor]]
false_outcome = df_binary[df_binary['cancer'] == False][[predictor]]
test_pos_rate = []
tp_rate = []
fp_rate = []

for (threshold,prevalence) in zip(thresholds,prevalence_values):
    count += 1
    #print('iteration: ' + str(count))
    
    test_pos_rate.append(pd.Series(df_binary[predictor] >= threshold).value_counts()[1]/750)
    
    #### Indexing [1] doesn't work w/ value_counts since only 1 index ([0]), so [1] returns an error
    #### Have to try/except this so that when indexing doesn't work, can input 0
    
    try:
        tp_rate.append(pd.Series(true_outcome[predictor] >= threshold).value_counts()[1]/len(true_outcome[predictor])*(prevalence))
    except KeyError:
        tp_rate.append(0/len(true_outcome[predictor])*(prevalence))
    
    try:
        fp_rate.append(pd.Series(false_outcome[predictor] >= threshold).value_counts()[1]/len(false_outcome[predictor])*(1-prevalence))
    except KeyError:
        fp_rate.append(0/len(false_outcome[predictor])*(1-prevalence))



In [564]:
df['tpr'] = tp_rate

In [566]:
df['fpr'] = fp_rate